In [1]:
import pandas as pd
import os
import numpy as np
import re
import glob
import matplotlib.pyplot as plt

In [2]:
pwd

'C:\\Users\\YUHA\\Desktop\\DScover\\사회2조 final project'

In [3]:
%cd data

C:\Users\YUHA\Desktop\DScover\사회2조 final project\data


# 1. 파일 합치기

In [4]:
keyword_list=glob.glob('0516*')
keyword_list

['0516_성매매유형', '0516_성매매행위', '0516_트렌드']

In [6]:
x=pd.DataFrame()
for k in keyword_list:
    file_list = os.listdir(k)
    
    for f_name in file_list:
        file_dir2 = '%s/%s' %(k, f_name) #파일 경로 설정
        a = pd.read_csv(file_dir2, encoding='CP949')
        m = f_name[0:-7] #검색어 : 'abc.csv'부분 삭제
        search_word = [m for i in range(len(a))] #파일 이름을 검색어 리스트로 생성
        a['search_word']=search_word #열 추가
        x = pd.concat([x, a])

In [7]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 257085 entries, 0 to 1270
Data columns (total 33 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 257085 non-null  float64
 1   conversation_id    257085 non-null  float64
 2   created_at         257085 non-null  object 
 3   date               257085 non-null  object 
 4   time               257085 non-null  object 
 5   timezone           257085 non-null  object 
 6   user_id            257085 non-null  float64
 7   username           257085 non-null  object 
 8   name               257085 non-null  object 
 9   tweet              257085 non-null  object 
 10  language           257084 non-null  object 
 11  mentions           257084 non-null  object 
 12  urls               257084 non-null  object 
 13  photos             257084 non-null  object 
 14  replies_count      257084 non-null  float64
 15  retweets_count     257084 non-null  float64
 16  like

In [8]:
x.to_csv('dataset1.csv', index=False, encoding='CP949')

# 2. 전처리

- [x] null 값 처리<br>
- [x] timezone, cashtags, retweet 열 삭제 <br>
- [x] data type 정리<br>
- [x] 명사 비율 열 추가<br>
- [x] 성매매광고트윗 판별 열(y) 추가<br>
- [ ] outlier 처리<br>

In [9]:
x = pd.read_csv('dataset1.csv', encoding='CP949', low_memory=False)

C:\Users\YUHA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### null 값 처리

In [10]:
x.isnull().sum() #null값 있는 열 확인

id                        0
conversation_id           0
created_at                0
date                      0
time                      0
timezone                  0
user_id                   0
username                  0
name                      0
tweet                     0
language                  1
mentions                  1
urls                      1
photos                    1
replies_count             1
retweets_count            1
likes_count               1
hashtags                  1
cashtags                  1
link                      1
retweet                   1
quote_url            256738
video                     1
thumbnail             73822
reply_to                  1
tweet_len                 0
nick_len                  0
nick_count_Q              0
tweet_count_Q             0
tweet_count_space         0
space_div_len             0
last_word                 0
search_word               0
dtype: int64

In [11]:
pd.set_option('display.max_columns', 100) #모든 열 보이기
x[x['link'].isnull()] #너무 많은 열에서 null인 행이 하나 있어서

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to,tweet_len,nick_len,nick_count_Q,tweet_count_Q,tweet_count_space,space_div_len,last_word,search_word
205147,1.380000e+18,1.380000e+18,2021-04-08 08:02:31 대한민국 표준시,2021-04-08,8:02:31,900,9.670000e+17,swcd0nkor50385d,럽셍,??광명애인대행 ??김포섹,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,2,0,4,1,0.071,False,콜걸


In [12]:
x = x.drop(index=205147) #삭제

In [41]:
x = x.reset_index() #인덱스 재설정

In [13]:
x[['quote_url','thumbnail']] = x[['quote_url','thumbnail']].fillna('[]') #null값 채우기

### 의미 없는 열 삭제

In [14]:
for i in range(32):
    print(x.columns[i], ':', x.iloc[:,i].nunique()) #각 열의 고유값 개수

id : 12472
conversation_id : 11753
created_at : 91323
date : 59
time : 55554
timezone : 3
user_id : 10051
username : 11677
name : 10693
tweet : 85451
language : 31
mentions : 43
urls : 562
photos : 71340
replies_count : 25
retweets_count : 106
likes_count : 128
hashtags : 1012
cashtags : 1
link : 111497
retweet : 1
quote_url : 325
video : 2
thumbnail : 71494
reply_to : 1910
tweet_len : 232
nick_len : 50
nick_count_Q : 34
tweet_count_Q : 48
tweet_count_space : 73
space_div_len : 377
last_word : 2


In [15]:
x = x.drop(['timezone','cashtags','retweet'], axis=1) #고유값이 하나인 열과 timezone 열 삭제

### data type 정리

In [16]:
x[['id', 'conversation_id','user_id']] = x[['id', 'conversation_id','user_id']].astype(str)
x[['replies_count','retweets_count','likes_count','video','last_word']] = x[['replies_count','retweets_count','likes_count','video','last_word']].astype(int)

### 명사 비율 열 추가

In [17]:
from konlpy.tag import Okt
from collections import Counter
okt=Okt() #사용 객체 선언

In [44]:
noun_div_pos = []
for i in range(len(x)):
    noun = okt.nouns(x['tweet'][i]) #명사 추출
    pos = okt.pos(x['tweet'][i]) #pos : part of speech 품사
    a = len(noun)/len(pos) #명사 비율
    noun_div_pos.append(a)
x['noun_div_pos'] = noun_div_pos

### 성매매광고트윗 판별 열(y) 추가

In [47]:
y=[]
#일반 트윗 검색어
trend = ['더블헤더', '미세먼지', '보건실침대', '수업시간몰폰', '어버이날', '피어리스', '허벅지씨름', '호흡곤란']
for word in x['search_word']:
    if (any(word in t for t in trend)):
        y.append(0)
    else:
        y.append(1)

In [48]:
x['y']=y

In [54]:
x.to_csv('성매매광고트윗_데이터셋.csv', index=False, encoding='CP949')

### oulier 처리

In [ ]:
#IQR 방식을 이용한 oulier index를 얻는 함수
def get_outlier(df=None, column=None, weight=1.5):
    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)
    
    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight
    
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
    
    outlier_idx = df[column][(df[column] < lowest)|df[column]>highest].index
    return outlier_idx